In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [2]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../../creds/'
creds_orch_file = "creds-orch.txt"

with open(os.path.join(path, creds_orch_file), "r") as myfile:
    usr_name = myfile.readline().strip()  # Read the first line and strip any leading/trailing whitespace
    tabExpTrackAPIKey = myfile.readline().strip()  # Read the second line and strip any leading/trailing whitespace

print('API Key Loaded!')

API Key Loaded!


In [3]:
ex = ExperimentClient(verbose = True)
ex.define_orch_cred(usr_name, tabExpTrackAPIKey)
ex.__dict__

{'expname': None,
 'model_groups': None,
 'data_groups': None,
 'applications': None,
 'run_id': None,
 'stuck_task': '',
 'prev_task': '',
 'prev_X': None,
 'prev_Y': None,
 'prev_categorical_indicator': None,
 'prev_attribute_names': None,
 'orchname': 'bart',
 'orchseceret': 'b8d206dce517c35e156cf7a5da01fc7b2fb095ae05028968ed6ba3be211df85f2dbc59b232bd53520206a1c73c65f14f9b266d9aeeadcc38e5a525c306e65628',
 'openMLAPIKey': None,
 'verbose': True,
 'suites_ids': [336, 335, 337, 334],
 'suites': [],
 'taskID_suite': [],
 'runs_per_pair': 60}

In [9]:
from requests.exceptions import RequestsDependencyWarning

expname = "08_multimodal7"

slice_start = 0
slice_size = 1000
i = 0
data = []
while True:
    url = ""
    payload = json.dumps({"experiment": expname, "slice_start": slice_start, "slice_end": slice_start+slice_size})
    headers = {'Name': usr_name,'Seceret': tabExpTrackAPIKey,'Content-Type': 'application/json'}
    t1 = time.time()
    resp_unparse = requests.request("POST", url, headers=headers, data=payload)
    t2=time.time()
    resp = json.loads(resp_unparse.text)
    t3=time.time()
    print('========= {} ========'.format(i))
    print('request time: {}'.format(t2-t1))
    print('parse time: {}'.format(t3-t2))
    print('length: {}'.format(len(resp)))
    data.extend(resp)
    if len(resp) == 0:
        break
    slice_start+=slice_size
    i+=1

print('total length: {}'.format(len(data)))

========= 0 ========
request time: 2.5504250526428223
parse time: 0.028622150421142578
length: 1000
========= 1 ========
request time: 1.649980068206787
parse time: 0.011717081069946289
length: 1000
========= 2 ========
request time: 1.9084820747375488
parse time: 0.036570072174072266
length: 1000
========= 3 ========
request time: 2.9910058975219727
parse time: 0.038848161697387695
length: 1000
========= 4 ========
request time: 1.8349800109863281
parse time: 0.16246414184570312
length: 1000
========= 5 ========
request time: 1.7958478927612305
parse time: 0.014723777770996094
length: 1000
========= 6 ========
request time: 2.0128228664398193
parse time: 0.04418587684631348
length: 1000
========= 7 ========
request time: 1.4594531059265137
parse time: 0.015451908111572266
length: 409
========= 8 ========
request time: 1.1514942646026611
parse time: 8.20159912109375e-05
length: 0
total length: 7409


In [14]:
len(data)

7409

In [15]:
df_model = [] # model
df_dataset = [] # task
df_hyperparameters = [] # hyp
df_metrics = []


for i, observation in enumerate(data):
    df_model.append(observation['model'])
    df_dataset.append(observation['task'])
    df_hyperparameters.append(observation['hyp'])
    try:
        df_metrics.append(observation['metrics_per_epoch'][-1])
    except:
        df_metrics.append(None)
else:
    sklearn_df = pd.DataFrame({
        'model': df_model,
        'dataset': df_dataset,
        'hyperparameters': df_hyperparameters,
        'metrics': df_metrics
    })

In [16]:
print(sklearn_df)

                      model     dataset   
0      SK_histGradBoost_reg  336-361083  \
1       SK_decisionTree_reg  336-361083   
2     rforestry_lintree_reg  336-361083   
3              CatBoost_reg  336-361083   
4                LGBM_class  334-361285   
...                     ...         ...   
7404           CatBoost_reg  335-361097   
7405    SK_decisionTree_reg  335-361097   
7406   SK_histGradBoost_reg  335-361097   
7407     rforestry_tree_reg  335-361097   
7408            XGBoost_reg  335-361097   

                                        hyperparameters   
0     {'learning_rate': 0.03461559161693327, 'max_de...  \
1     {'max_depth': None, 'criterion': 'squared_erro...   
2     {'mtry_fraq': 0.37995388058270396, 'nodesize_s...   
3     {'learning_rate': 0.7803083370088788, 'random_...   
4     {'n_estimators': 92, 'learning_rate': 0.100198...   
...                                                 ...   
7404  {'learning_rate': 0.2959279444789412, 'random_...   
7405  {'max

In [17]:
sklearn_df.to_csv('MultiModalResults.csv', index=False)